##**1. Entendimento do Contexto / Negócio**

**ESPAÇO RESERVADO PARA TEXTO SOBRE O CONTEXTO**

##**2. Entendimento inicial dos Dados**

### **Instalações e Bibliotecas necessárias**

In [83]:
"""
ftfy (fixes text for you) é uma biblioteca Python que ajuda a corrigir e limpar texto codificado de forma incorreta,
seja devido a problemas de codificação, caracteres inválidos ou outros problemas semelhantes. Ele oferece uma variedade
de funções para corrigir e normalizar texto, incluindo a detecção e correção de codificações ruins, a remoção de
caracteres inválidos e a padronização de espaços em branco.

Instalação:
    Para instalar o ftfy via pip, você pode usar o seguinte comando:

    !pip install ftfy

Referência:
    Documentação oficial do ftfy: https://ftfy.readthedocs.io/en/latest/
"""

!pip install ftfy

In [84]:
!pip install dash

In [120]:
"""
Importações de bibliotecas comumente usadas em análise de dados e visualização.

Este bloco de importações inclui as seguintes bibliotecas:
- requests: para fazer solicitações HTTP.
- pandas: para manipulação e análise de dados em formato tabular.
- StringIO da io: para manipulação de strings como arquivos.
- tabulate: para exibir dados tabulares em uma formatação legível.
- plotly.express: para visualizações interativas e expressivas.
- make_subplots de plotly.subplots: para criar subtramas em visualizações plotly.
- go de plotly.graph_objects: para criar figuras e gráficos plotly de maneira programática.
- numpy: para operações numéricas eficientes.
- matplotlib.pyplot: para visualizações estáticas e personalizáveis.
- seaborn: para visualizações estatísticas atraentes e informativas.
- scipy.stats: para funções estatísticas e testes de hipóteses.

Referência:
    Documentação oficial das bibliotecas individuais para mais detalhes sobre suas funcionalidades e uso.
"""

import requests
import pandas as pd
from io import StringIO
import io
import ftfy
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# from tabulate import tabulate



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from scipy.stats import ttest_ind

###**Extração e leitura dos dados**

In [86]:
"""
Configura as opções de exibição do pandas para mostrar um número máximo de colunas, linhas e largura de coluna ilimitados.

Referência:
    Documentação oficial do pandas sobre opções de exibição: https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [87]:
"""
Este bloco de código baixa um arquivo CSV de uma URL, decodifica seu conteúdo, e o converte em um DataFrame pandas.
Em seguida, realiza um tratamento na coluna "Endereço" para remover caracteres de nova linha.
Por fim, exibe as primeiras linhas do DataFrame.
"""

file_url = "https://github.com/Ada-Empregabilidade/adahack-2024-dados/raw/main/base_dados/base_de_dados.csv"

response = requests.get(file_url)

if response.status_code == 200:
    decoded_content = ftfy.fix_text(response.content.decode('utf-8'))

    df = pd.read_csv(io.StringIO(decoded_content))

    df['Endereço'] = df['Endereço'].str.replace('\n', ' ')

    display(df.head())
else:
    print(f"Erro ao obter o arquivo: {response.status_code}")


,id,id.1,Nome,Genero,Idade,Raça,Endereço,Formação,Estado,Tempo de casa,Departamento,Senioridade
0,1,1,Eleonora Arilda Penedo Gomes de Padilha,Fem,34.0,pardo,"9155 Harold Oval Sellersside, FL 21337",Ensino Médio,Santa Catarina,12.0,Compras,Analista Pleno
1,2,2,Elisângela Gabrielle de Osório,Fem,26.0,pardo,"941 Martin Manor Lake Isaiahtown, FM 43797",Ensino Médio,Pará,6.0,Contabilidade,Analista Júnior
2,3,3,José Túlio de Cabral,Masc,35.0,pardo,"110 Davis Ridges Mejiaville, LA 17095",Ensino Médio,Santa Catarina,5.0,Vendas,Analista Pleno
3,4,4,Ezequiel Edivaldo de Medeiros Sonao,Masc,24.0,pardo,"48010 Wilson Glen Apt. 749 Smithborough, NV 00864",Ensino Superior,Tocantins,4.0,Administrativo,Gerente
4,5,5,Fagner Josiel dos Santos,Masc,21.0,pardo,"8666 Ramos Ports Apt. 070 Sandraport, MN 33570",Ensino Superior,Ceará,5.0,Recursos Humanos,Analista Júnior


###**Colunas do DataFrame que não agregam valor**
Após análise, foram identificadas colunas que não agregam valor, que serão excluídas do DataFrame.

In [88]:
"""
Excluindo colunas do DataFrame que não agregam valor
"""
df = df.drop(columns=['id', 'id.1', 'Endereço'])

###**Quantidade de linhas e colunas**

In [89]:
"""
Obtendo a quantidade de linhas e colunas do dataframe criado a partir da extração dos dados
"""
num_linhas = len(df)
num_colunas = len(df.columns)

print(f"Número de Linhas: {num_linhas}")
print(f"Número de Colunas: {num_colunas}")

Número de Linhas: 10000
Número de Colunas: 9


###**Verificação dos tipos das colunas**

In [90]:
"""
Verificação dos tipos das colunas existentes (int64, object, float)
"""
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Nome           10000 non-null  object 
 1   Genero         10000 non-null  object 
 2   Idade          9944 non-null   float64
 3   Raça           10000 non-null  object 
 4   Formação       9956 non-null   object 
 5   Estado         10000 non-null  object 
 6   Tempo de casa  9800 non-null   float64
 7   Departamento   10000 non-null  object 
 8   Senioridade    10000 non-null  object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


###**Verificação de valores duplicados - Geral**

In [91]:
duplicados = df.duplicated().sum()
print(f"Existem {(duplicados)} dados duplicados")

Existem 0 dados duplicados


###**Verificação de valores únicos**

In [92]:
def summarize_dataframe(df):
    """
    Sumariza um DataFrame, fornecendo informações sobre os valores únicos em cada coluna.

    Parâmetros:
        df (DataFrame): O DataFrame a ser sumarizado.

    Retorna:
        DataFrame: Um DataFrame contendo informações sobre os valores únicos em cada coluna do DataFrame de entrada.
                   O DataFrame resultante tem três colunas: 'Coluna', 'Valores Únicos' e 'Número de Valores Únicos'.
    """
    dfs = []

    for column in df.columns:
        unique_values = df[column].unique()
        num_unique_values = len(unique_values)
        temp_df = pd.DataFrame({'Coluna': [column], 'Valores Únicos': [unique_values], 'Número de Valores Únicos': [num_unique_values]})
        dfs.append(temp_df)

    df_output = pd.concat(dfs, ignore_index=True)

    return df_output

#Chamando a função e e imprimindo dados
df_summary = summarize_dataframe(df)
display(df_summary)

,Coluna,Valores Únicos,Número de Valores Únicos
0,Nome,"[Eleonora Arilda Penedo Gomes de Padilha, Elisângela Gabrielle de Osório, José Túlio de Cabral, Ezequiel Edivaldo de Medeiros Sonao, Fagner Josiel dos Santos, Magali Luzimara da Silva, Victor Plínio Shufrouze, Laércio Rossi, Francisco Ângelo de Toledo Seixas, Fabrício Rangel de Gonçalves dos Santos, Laila da Silva, Nivaldo Ademar Valverde dos Santos, Mariano Rosa da Graça Neto, Eulália Brasil de Bezerra, Marlon Rafael do Nascimento Uason, César da Paraíba, Toninho Nivaldo Siato, Fabiano Táles Negrão, Adônis Otto de Aguiar da Lira, Álvaro Cristiano Schmidt, Arthur Uckimovic Júnior, Emílio Bahia, Mariano de Angola Onmmylte Neto, Milton Eugênio Shetum de Figueira, Ricardo Diogo da Luz Nile Neto, Aline de Uchôa, Heloísa Laura Eala Aisvic, Henrique Inácio Lussolson Tegego, Milene Thaís Aciz, Patrícia de Miranda Xizu, Valdirene Nunes, Rafaela Paim do Amaral, Berenice Tono, Juliano d'Ávila Júnior, Yuri Góis Dutra, Edson Felipe Ieldi Girru, Ali Túlio de Miranda, Ayrton Ricardo do Espírito Santo Gomes, Sâmia Juliana do Espírito Santo de Tavares, Zuleide Castelo, Renata Maria de Espanha Chearndeã do Amaral, Cléber dos Santos de Carvalho, Fabiano Gomes, Roberval Osvaldo Pyva Esalicason de Pinheiro, Junior de Souza Abon, Luan de Brito, Leandro Roberval Tyuslean, Nicolas de Pimentel Etozvic Aodintom, Jacinto de Azevedo da Silva, Albino da Silva dos Santos, Dilma Berenice Vyratu Rage de Barbosa, Joel Marlon Hihopi, Francisco Ysemelman Liyrn, Andréia Rebeca da Silva, Wendy Genir de Ortiz Pinto da Silva, Arnaldo de Lemos, Inácio dos Santos Iprijofez de Oliveira Neto, Flávio Lúcio Schneider, Zeca Antônio da Nóbrega Weber Ezze, Diogo Sérgio de Assis, Bianca Amália Espada Adoan, Sérgio Nilo de Arantes Bopi Júnior, Arthur Alexandre de Souza Marques, Sebastião de Muniz Clyloier, Martinho Arnaldo da Silva Dutra, Antônio de Souza Uji Júnior, Hélio Ademar Gryamman Júnior, Mikaella Berenice de Lisboa, Eric de Freitas, Edson Nicolau Marques, Kevin Toztin de Sobral, Olga de Drummond, Diogo de São Pedro Pultonão Negrão, Nilo Patrick de Mendonça, Rogério Nelson Vigrylaz, Reinaldo Osório da Luz Milani de Magalhães, Júlio Edivaldo de Alencar, Junior Edinaldo de Aguiar, Kim Roberval de Lisboa, Altair de Sá dos Santos Júnior, Eduardo Mohoite dos Santos, Isabela de Borges, Jean Eugarnson Gomes, Edu de Drummond, Liane da Gama de Amorim, Everton Paim Piwa, Graziele Albina Rosatto, Jean Cauê de Macedo, Wanda de Malta, Márcia de Barbosa Yazeze, Ariana Quésia Buarque de Sales Noshiguke, Elói Joel Nugi Odrean, Patrick de Machado Iloltri Cavalcante Terceiro, Melanie Evelyn Fifenman, Magali Zara Lopes, Rafael dos Santos, Kim de Moreira, Everaldo Denis Uzbenseã Júnior, Yuri Marco de Tavares Woibi, Eliane Márcia da Graça Esery de Moreira, ...]",9961
1,Genero,"[Fem, Masc]",2
2,Idade,"[34.0, 26.0, 35.0, 24.0, 21.0, 49.0, 29.0, 40.0, 33.0, 54.0, 18.0, 19.0, 37.0, 38.0, 41.0, 46.0, 31.0, 32.0, 48.0, 39.0, 43.0, 44.0, 47.0, 27.0, 28.0, 30.0, 22.0, 20.0, 36.0, 25.0, 42.0, 53.0, 56.0, 23.0, 45.0, 52.0, 50.0, 51.0, nan, 16.0, 58.0, 68.0, 17.0, 57.0, 55.0, 12.0, 63.0, 15.0, 10.0, 61.0, 65.0, 6.0, 13.0, 64.0, 11.0, 14.0, 60.0, 62.0, 9.0, 5.0, 8.0, 74.0]",62
3,Raça,"[pardo, preto, branco, indígena, amarelo]",5
4,Formação,"[Ensino Médio, Ensino Superior, Pós graduação, Mestrado, Doutorado, nan]",6
5,Estado,"[Santa Catarina, Pará, Tocantins, Ceará, São Paulo, Rondônia, Amapá, Minas Gerais, Acre, Roraima, Amazonas, Paraná, Bahia, Piauí, Goiás, Rio de Janeiro, Rio Grande do Norte, Pernambuco, Mato Grosso, Rio Grande do Sul, Espírito Santo, Maranhão, Paraíba, Distrito Federal, Mato Grosso do Sul, Sergipe, Alagoas]",27
6,Tempo de casa,"[12.0, 6.0, 5.0, 4.0, 3.0, 9.0, 8.0, 1.0, 0.0, 7.0, nan, 2.0, 10.0, -2.0, 11.0, 14.0, -3.0, 13.0, -1.0, 15.0, -5.0, -6.0, 17.0, 16.0]",24
7,Departamento,"[Compras, Contabilidade, Vendas, Administrativo, Recursos Humanos, Operações, Desenvolvimento de Produtos, Marketing, TI]",9
8,Senioridade,"[Analis

###**Verificação de valores ausentes**

In [93]:
def group_columns_by_percentile(df):
    """
    Agrupa as colunas de um DataFrame por faixa percentual de valores faltantes.

    Parâmetros:
        df (DataFrame): O DataFrame a ser agrupado.

    Retorna:
        DataFrame: Um DataFrame contendo as colunas agrupadas por faixa percentual de valores faltantes,
                   juntamente com o percentual exato de valores faltantes em cada coluna.
                   O DataFrame resultante possui três colunas: 'Faixa Percentual', 'Colunas' e 'Percentual Exato'.
    """

    colunas_por_faixa = {}
    for i in range(20):
        inicio = f'{i * 5:02.0f}.01' if i == 0 else f'{i * 5:02.0f}.01'
        fim = f'{(i + 1) * 5 - 0.01:02.0f}'
        colunas_por_faixa[f'{inicio}% até {fim}%'] = []

    colunas_por_faixa['0.00% (zero erro)'] = []

    colunas_por_percentual = {}

    for coluna in df.columns:
        percentual_faltante = df[coluna].isnull().sum() / len(df) * 100
        if percentual_faltante == 0:
            colunas_por_faixa['0.00% (zero erro)'].append(coluna)
            colunas_por_percentual[coluna] = 0.00
        else:
            for i in range(20):
                inicio = f'{i * 5:02.0f}.01' if i == 0 else f'{i * 5:02.0f}.01'
                fim = f'{(i + 1) * 5 - 0.01:02.0f}'
                faixa = f'{inicio}% até {fim}%'
                if percentual_faltante <= (i + 1) * 5:
                    colunas_por_faixa[faixa].append(coluna)
                    colunas_por_percentual[coluna] = round(percentual_faltante, 2)
                    break
            if percentual_faltante > 95:
                colunas_por_faixa['95.01% até 100%'].append(coluna)
                colunas_por_percentual[coluna] = round(percentual_faltante, 2)

    colunas_por_faixa = {'0.00% (zero erro)': colunas_por_faixa['0.00% (zero erro)']} | {k: v for k, v in colunas_por_faixa.items() if k != '0.00% (zero erro)'}

    df_tabela = pd.DataFrame(colunas_por_faixa.items(), columns=['Faixa Percentual', 'Colunas'])
    df_tabela['Percentual Exato'] = df_tabela['Colunas'].apply(lambda colunas: [colunas_por_percentual[coluna] for coluna in colunas])

    return df_tabela

#Chamando a função e e imprimindo dados
df_grouped = group_columns_by_percentile(df)
display(df_grouped)

,Faixa Percentual,Colunas,Percentual Exato
0,0.00% (zero erro),"[Nome, Genero, Raça, Estado, Departamento, Senioridade]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,00.01% até 05%,"[Idade, Formação, Tempo de casa]","[0.56, 0.44, 2.0]"
2,05.01% até 10%,[],[]
3,10.01% até 15%,[],[]
4,15.01% até 20%,[],[]
5,20.01% até 25%,[],[]
6,25.01% até 30%,[],[]
7,30.01% até 35%,[],[]
8,35.01% até 40%,[],[]
9,40.01% até 45%,[],[]


####**Coluna -> "Formação"**

In [94]:
df['Formação'].isnull().sum()

44

In [95]:
registros_nulos_formacao = df[df['Formação'].isnull()]
display(registros_nulos_formacao)

,Nome,Genero,Idade,Raça,Formação,Estado,Tempo de casa,Departamento,Senioridade
414,Sandro Fagner Dutra dos Santos da Silva,Masc,16.0,branco,NaN,Acre,7.0,Recursos Humanos,Gerente
660,Jack Fiyita,Masc,17.0,branco,NaN,Alagoas,7.0,Compras,Analista Pleno
1227,João de França dos Reis do Nascimento,Masc,12.0,branco,NaN,Rio de Janeiro,8.0,Compras,Analista Júnior
1554,Reginaldo de Mello da Costa Júnior,Masc,16.0,branco,NaN,Minas Gerais,-3.0,Administrativo,Gerente
1672,Gabriel Nicolau Rangel,Masc,17.0,branco,NaN,São Paulo,6.0,TI,Gerente
1676,Eric Victor de Garcia,Masc,16.0,branco,NaN,Distrito Federal,4.0,Desenvolvimento de Produtos,Analista Sênior
1927,Eliane Shoa Trevisan,Fem,15.0,branco,NaN,Roraima,0.0,Vendas,Diretor
1981,Bartolomeu Iago de França Iusov,Masc,15.0,branco,NaN,Acre,9.0,Contabilidade,Analista Júnior
2396,Raí da Silva,Masc,10.0,branco,NaN,Pernambuco,10.0,Marketing,Analista Sênior
2612,Charles de Viana da Silva,Masc,10.0,branco,NaN,Paraíba,7.0,Recursos Humanos,Estagiário


In [96]:
"""
Excluindo valores ausentes da coluna 'Formação' do DataFrame
"""
df = df.dropna(subset=['Formação'])

####**Coluna -> "Tempo de casa"**

In [97]:
# Quantidade de valores negativos na coluna 'Tempo de casa'
quantidade_negativos_tempo_de_casa = (df['Tempo de casa'] < 0).sum()
print(quantidade_negativos_tempo_de_casa)

97


In [98]:
# Quantidade de valores NaN na coluna 'Tempo de casa'
quantidade_nan_tempo_de_casa = df['Tempo de casa'].isnull().sum()
print(quantidade_nan_tempo_de_casa)

199


In [99]:
quantidade_negativos_nan_tempo_de_casa = (df['Tempo de casa'] < 0).sum() + df['Tempo de casa'].isnull().sum()
display(quantidade_negativos_nan_tempo_de_casa)

296

In [100]:
quantidade_negativos_nan_tempo_de_casa = df[(df['Tempo de casa'] < 0) | (df['Tempo de casa'].isnull())]
display(quantidade_negativos_nan_tempo_de_casa)

,Nome,Genero,Idade,Raça,Formação,Estado,Tempo de casa,Departamento,Senioridade
30,Valdirene Nunes,Fem,32.0,pardo,Ensino Superior,Rio Grande do Norte,NaN,Marketing,Analista Júnior
33,Juliano d'Ávila Júnior,Masc,43.0,branco,Ensino Superior,Paraná,NaN,Marketing,Analista Júnior
62,Arthur Alexandre de Souza Marques,Masc,25.0,pardo,Ensino Superior,Paraná,-2.0,Desenvolvimento de Produtos,Analista Sênior
90,Ariana Quésia Buarque de Sales Noshiguke,Fem,31.0,pardo,Ensino Superior,Rio Grande do Sul,NaN,Compras,Gerente
105,Adriano Uribe Irnizson Imekre Clozklo,Masc,39.0,preto,Ensino Superior,Rio Grande do Sul,-3.0,Marketing,Analista Pleno
118,Verônica Betayomo da Silva Umriov,Fem,35.0,pardo,Mestrado,Rio de Janeiro,-2.0,Vendas,Analista Júnior
160,João Rosatto do Amaral,Masc,19.0,pardo,Ensino Superior,Pará,NaN,Operações,Estagiário
173,Paulo Alexandre da Silva,Masc,32.0,pardo,Ensino Superior,Sergipe,NaN,Vendas,Analista Sênior
193,Jorge Edinaldo de Lins,Masc,43.0,preto,Pós graduação,Rio de Janeiro,NaN,Compras,Gerente
203,Altino Nelson da Rocha Fiacaz,Masc,27.0,pardo,Doutorado,Goiás,NaN,Contabilidade,Analista Júnior


In [101]:
# Excluir valores negativos e valores ausentes da coluna 'Tempo de casa'
df = df[(df['Tempo de casa'] >= 0) & (~df['Tempo de casa'].isnull())]

####**Coluna -> "Idade"**

In [102]:
df['Idade'].isnull().sum()

55

In [103]:
registros_nulos_idade = df[df['Idade'].isnull()]
display(registros_nulos_idade)

,Nome,Genero,Idade,Raça,Formação,Estado,Tempo de casa,Departamento,Senioridade
361,Alisson de Toledo Júnior,Masc,NaN,pardo,Ensino Superior,Rondônia,4.0,Vendas,Analista Júnior
458,Toninho de Mendonça Opewo,Masc,NaN,pardo,Ensino Superior,Rio Grande do Sul,5.0,Vendas,Estagiário
635,Natália Gabriela de Pascoal,Fem,NaN,pardo,Ensino Médio,Ceará,9.0,Recursos Humanos,Analista Pleno
783,Cícera de Souza,Fem,NaN,pardo,Ensino Médio,Paraná,4.0,TI,Analista Sênior
1057,Willian Edivaldo Uzilov Rosatto,Masc,NaN,pardo,Ensino Superior,Roraima,7.0,Vendas,Analista Pleno
1065,Samuel Cristiano da Silva Olebuão,Masc,NaN,pardo,Ensino Superior,Paraíba,7.0,Contabilidade,Analista Pleno
1114,Lino Odurro Cispeson,Masc,NaN,pardo,Ensino Superior,Santa Catarina,4.0,TI,Analista Pleno
1504,Jairo Cícero Olshasian,Masc,NaN,pardo,Ensino Superior,Rio Grande do Sul,6.0,Recursos Humanos,Analista Pleno
1677,Yasmin Nádia da Rocha,Fem,NaN,pardo,Ensino Superior,Ceará,6.0,Contabilidade,Gerente
1710,Mike Ywuz Branco,Masc,NaN,pardo,Ensino Médio,Mato Grosso,5.0,Recursos Humanos,Gerente


In [104]:
"""
Excluindo valores ausentes da coluna 'Idade' do DataFrame
"""
df = df.dropna(subset=['Idade'])

###**Verificação de valores duplicados - Por coluna**

####**Coluna -> "Nome"**

In [105]:
registros_duplicados_nome = df[df.duplicated(subset=['Nome'], keep=False)]
registros_duplicados = registros_duplicados_nome.sort_values(by='Nome')
display(registros_duplicados_nome)

,Nome,Genero,Idade,Raça,Formação,Estado,Tempo de casa,Departamento,Senioridade
192,Jenny de Godoy,Fem,42.0,branco,Ensino Superior,Rio de Janeiro,7.0,Vendas,Analista Sênior
234,Cristiano da Silva,Masc,20.0,branco,Ensino Superior,Sergipe,4.0,Vendas,Analista Júnior
592,Martin de Souza,Masc,39.0,branco,Ensino Superior,Roraima,7.0,Administrativo,Analista Sênior
594,Lina dos Santos,Fem,42.0,pardo,Mestrado,Bahia,4.0,Desenvolvimento de Produtos,Gerente
681,Luzimara da Silva,Fem,35.0,pardo,Ensino Superior,Mato Grosso,0.0,Administrativo,Analista Sênior
914,Cleberson de Oliveira,Masc,33.0,preto,Ensino Superior,Minas Gerais,4.0,Vendas,Analista Sênior
920,Reginaldo dos Santos,Masc,32.0,pardo,Ensino Médio,São Paulo,10.0,Contabilidade,Analista Sênior
928,Aécio de Souza,Masc,32.0,branco,Ensino Superior,Paraná,4.0,Administrativo,Gerente
1023,Augusto de Souza,Masc,42.0,branco,Ensino Superior,Santa Catarina,8.0,Compras,Analista Júnior
1448,Carolina de Souza,Fem,18.0,preto,Ensino Superior,Espírito Santo,4.0,Desenvolvimento de Produtos,Analista Júnior


In [106]:
len(registros_duplicados_nome)

74

###**Relatório 1 - Considerações iniciais**

Nesta seção, buscou-se observar algumas características gerais dos dados.

O conjunto original dos dados possui 10000 registros e 12 colunas.

Algumas verificações foram realizadas para garantir dados de qualidade nas seções subsequentes.

**1) Quanto à dados que não agregam valor**
>Na **1.1) verificação dos dados que não agregam valor**, observou-se que as colunas:
>>**a) id**, **id.1** e **Endereço**: Não agregam valor para os objetivos desta análise e, portanto, foram excluídas do DataFrame.

**2) Quanto ao tipo dos dados:**
>Na **2.1) verificação dos tipos das colunas**, observou-se que a coluna:
>>**a) Idade** e **Tempo de casa**: Inicialmente, causam estranheza por estarem em **float**. A priori, assume-se que o tipo mais adequado nesse caso seria **int64**, caso sejam utilizadas.**(vide Seção 3. Preparação dos Dados, Tratamento 1 para informação complementar**);

**3) Quanto a duplicidade dos dados - Geral:**
>Na **3.1) verificação de valores duplicados - Geral**, não foi encontrada duplicidade de registros.

**4) Quanto aos valores únicos:**
>Na **4.1) verificação de valores únicos**, ficou evidente que a coluna:
>>**a) Nome**: Possui 39 valores valores repetidos, pois existem 9961 valores únicos de um total de 10000 registros (**vide item 6.1) a) 👇 para informação complementar**);

>>**b) Formação**: Possui valores do tipo **NaN (vide item 5.1) a) 👇 para informação complementar**);

>>**c) Tempo de casa**: Possui valores do tipo **NaN** e valores **Negativos (vide item 5.1) b) 👇 para informação complementar**);

>>**d) Senioridade**: Possui valores com o mesmo significado, mas cadastrados com grafia incorreta **gerente** e **Gerente**

>>**e) Idade**: Possui valores do tipo **NaN**

**5) Quanto aos valores ausentes:**
>Na **5.1) verificação de valores ausentes ou negativos**, ficou evidente que na coluna:
>>**a) Formação**: Os 44 valores do tipo 'NaN' apresentaram muita inconsistência quando analisados em conjunto com as demais colunas. Haviam muitas situações como, por exemplo, uma pessoa muito jovem (menor de 18 anos) possuia um tempo de casa próximo de sua idade e com uma Senioridade muito elevada. Certamente, esses dados não estão confiáveis e, portanto, foram excluídos. Dimensionamos que isso impacta muito pouco nossas análises, pois corresponde a, apenas, 0.44% de todo o conjunto de dados.

>>**b) Tempo de casa**: Os 97 valores resmanescentes do tipo 'NaN' e os 199 valores 'Negativos' dessa coluna, juntos, totalizam 2.96% de todo os registros do conjunto de dados. Como esses valores representam baixo impacto, esses registros foram excluídos.

>>**c) Idade**: Os 56 valores remanescentes do tipo 'NaN' dessa coluna atinge 0.56% dos registros de todo o conjunto de dados. Como esses valores representam baixo impacto, esses registros foram excluídos.

**6) Quanto a duplicidade dos dados - Por coluna:**
>Na **6.1) verificação de valores duplicados - Por coluna**, ficou evidente que na coluna:
>>**a) Nome**: Os 37 valores remascentes repetidos da coluna não tratam-se de duplicidade de registros, portanto, não precisam ser removidos.

Ao todo, foram excluídos 395 registros, o equivalente a 3.95% de toda o conjunto dos dados.

Na Seção seguinte, avançamos na aplicação dos tratamentos identificados, após essas verificações iniciais.

##**3. Preparação dos Dados**

Na Seção anterior, foi feito o entendimento inicial dos dados e, na medida do necessário, algumas exclusões foram aplicadas, conforme explicações do Relatório 1, naquela Seção.

Com isso, nesta seção serão aplicados os tratamentos necessários remanescentes, a saber:
>

*   **Tratamento 1**: Transformação do tipo das colunas **Idade** e **Tempo de casa (vide item 2.1) a) ☝️ para informação complementar)**

*   **Tratamento 2**: Transformação do tipo 'replace' para padronizar valor 'gerente' (primeira letra em minúsculo) para 'Gerente' (primeira letra em maiúsculo **(vide item 4.1) d) ☝️ para informação complementar)**



###**Tratamento 1:** Transformação do tipo das colunas **Idade** e **Tempo de casa** para 'int64'

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9605 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Nome           9605 non-null   object 
 1   Genero         9605 non-null   object 
 2   Idade          9605 non-null   float64
 3   Raça           9605 non-null   object 
 4   Formação       9605 non-null   object 
 5   Estado         9605 non-null   object 
 6   Tempo de casa  9605 non-null   float64
 7   Departamento   9605 non-null   object 
 8   Senioridade    9605 non-null   object 
dtypes: float64(2), object(7)
memory usage: 750.4+ KB


In [108]:
df['Idade'] = df['Idade'].astype(int)
df['Tempo de casa'] = df['Tempo de casa'].astype(int)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9605 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nome           9605 non-null   object
 1   Genero         9605 non-null   object
 2   Idade          9605 non-null   int64 
 3   Raça           9605 non-null   object
 4   Formação       9605 non-null   object
 5   Estado         9605 non-null   object
 6   Tempo de casa  9605 non-null   int64 
 7   Departamento   9605 non-null   object
 8   Senioridade    9605 non-null   object
dtypes: int64(2), object(7)
memory usage: 750.4+ KB


###**Tratamento 2:** Transformação do tipo 'replace' para padronizar valor 'gerente' (primeira letra em minúsculo) para 'Gerente' (primeira letra em maiúsculo) na coluna **Senioridade**

In [110]:
df['Senioridade'] = df['Senioridade'].replace('gerente', 'Gerente')

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9605 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nome           9605 non-null   object
 1   Genero         9605 non-null   object
 2   Idade          9605 non-null   int64 
 3   Raça           9605 non-null   object
 4   Formação       9605 non-null   object
 5   Estado         9605 non-null   object
 6   Tempo de casa  9605 non-null   int64 
 7   Departamento   9605 non-null   object
 8   Senioridade    9605 non-null   object
dtypes: int64(2), object(7)
memory usage: 750.4+ KB


##**4. Análise Exploratória dos Dados**

###**Gênero**
Observando os dados sob a perspectiva de gênero para analisar equidade, igualdade e inclusão.

####**Distribuição de Gênero - Geral**

In [239]:
import plotly.graph_objects as go

# Contagem por gênero
counts_gender = df['Genero'].value_counts()
fig_gender = go.Figure()
fig_gender.add_trace(go.Bar(x=counts_gender.index, y=counts_gender.values, marker=dict(color=['blue', 'pink'])))

# Adicionar os valores de cada barra em cima delas
for i, count in enumerate(counts_gender.values):
    fig_gender.add_annotation(x=counts_gender.index[i], y=count, text=str(count), showarrow=False, font=dict(color='black', size=12), xshift=0, yshift=5)

fig_gender.update_layout(title='Distribuição por Gênero')

# Exibir o gráfico de distribuição de gênero
fig_gender.show()

####**Distribuição de Gênero por Formação**

In [210]:
import plotly.graph_objects as go
import math

# Distribuição por Formação
fig_education = go.Figure()

# Calculando as contagens de formação para cada gênero
female_education_counts = df[df['Genero'] == 'Fem']['Formação'].value_counts()
male_education_counts = df[df['Genero'] == 'Masc']['Formação'].value_counts()

# Obtendo o maior valor após o valor máximo observado na contagem de formação
max_value = max(female_education_counts.max(), male_education_counts.max())
max_y = math.ceil(max_value / 500) * 500

# Adicionando os histogramas de distribuição por formação para os gêneros Feminino e Masculino
fig_education.add_trace(go.Bar(x=female_education_counts.index, y=female_education_counts, name='Fem', marker=dict(color='pink'), text=female_education_counts, textposition='auto'))
fig_education.add_trace(go.Bar(x=male_education_counts.index, y=male_education_counts, name='Masc', marker=dict(color='blue'), text=male_education_counts, textposition='auto'))

# Atualizando o layout para mostrar as barras lado a lado e ajustar o eixo y
fig_education.update_layout(title='Distribuição por Formação', barmode='group', xaxis_title='Formação', yaxis_title='Contagem', yaxis=dict(range=[0, max_y], dtick=500))

# Exibindo o gráfico
fig_education.show()

####**Distribuição de Gênero por Tempo de casa - agrupado por faixas de períodos**

In [207]:
import pandas as pd
import plotly.graph_objects as go

# Definindo os limites dos grupos de tempo de casa
grupos_tempo_de_casa = [
    ('Até 2 anos', 2),
    ('3-5 anos', 5),
    ('6-10 anos', 10),
    ('11-15 anos', 15),
    ('16-20 anos', 20),
    ('21-25 anos', 25),
    ('26-30 anos', 30),
    ('31-35 anos', 35),
    ('Mais de 36 anos', float('inf'))
]

# Ordenando os grupos de tempo de casa em ordem crescente
grupos_tempo_de_casa = sorted(grupos_tempo_de_casa, key=lambda x: x[1])

# Criando uma função para categorizar o tempo de casa em grupos
def categorizar_tempo_de_casa(tempo):
    for nome_grupo, limite_superior in grupos_tempo_de_casa:
        if tempo <= limite_superior:
            return nome_grupo

# Aplicando a função para criar uma nova coluna 'Grupo de Tempo de Casa'
df['Grupo de Tempo de Casa'] = df['Tempo de casa'].apply(categorizar_tempo_de_casa)

# Criando uma série pandas com todos os grupos
todos_grupos = pd.Series([grupo[0] for grupo in grupos_tempo_de_casa])

# Calculando as contagens de tempo de casa para cada gênero
female_tempo_counts = df[df['Genero'] == 'Fem']['Grupo de Tempo de Casa'].value_counts()
male_tempo_counts = df[df['Genero'] == 'Masc']['Grupo de Tempo de Casa'].value_counts()

# Mesclando com a série de todos os grupos e preenchendo os valores ausentes com 0
female_tempo_counts = female_tempo_counts.reindex(todos_grupos, fill_value=0)
male_tempo_counts = male_tempo_counts.reindex(todos_grupos, fill_value=0)

# Criando o gráfico de barras
fig_tempo_casa = go.Figure()

# Adicionando os histogramas de distribuição por tempo de casa para os gêneros Feminino e Masculino
fig_tempo_casa.add_trace(go.Bar(x=female_tempo_counts.index, y=female_tempo_counts, name='Fem', marker=dict(color='pink'), text=female_tempo_counts, textposition='auto'))
fig_tempo_casa.add_trace(go.Bar(x=male_tempo_counts.index, y=male_tempo_counts, name='Masc', marker=dict(color='blue'), text=male_tempo_counts, textposition='auto'))

# Atualizando o layout do gráfico
fig_tempo_casa.update_layout(title='Distribuição por Tempo de Casa', barmode='group', xaxis_title='Grupo de Tempo de Casa', yaxis_title='Contagem', yaxis=dict(range=[0, 4500], dtick=500), xaxis=dict(categoryorder='array', categoryarray=[grupo[0] for grupo in grupos_tempo_de_casa]))

# Exibindo o gráfico
fig_tempo_casa.show()

####**Distribuição de Gênero por Departamento - Opção A**

In [209]:
import math
import plotly.graph_objects as go

# Distribuição por Departamento
fig_department = go.Figure()

# Calculando as contagens de departamento para cada gênero
female_department_counts = df[df['Genero'] == 'Fem']['Departamento'].value_counts()
male_department_counts = df[df['Genero'] == 'Masc']['Departamento'].value_counts()

# Obtendo o maior valor após o valor máximo observado na coluna 'Departamento'
max_value = max(female_department_counts.max(), male_department_counts.max())
max_y = math.ceil(max_value / 500) * 500

# Adicionando os histogramas de distribuição por departamento para os gêneros Feminino e Masculino
fig_department.add_trace(go.Bar(x=female_department_counts.index, y=female_department_counts, name='Fem', marker=dict(color='pink'), text=female_department_counts, textposition='auto'))
fig_department.add_trace(go.Bar(x=male_department_counts.index, y=male_department_counts, name='Masc', marker=dict(color='blue'), text=male_department_counts, textposition='auto'))

# Atualizando o layout para mostrar as barras lado a lado e ajustar o eixo y
fig_department.update_layout(title='Distribuição por Departamento', barmode='group', xaxis_title='Departamento', yaxis_title='Contagem', yaxis=dict(range=[0, max_y], dtick=500))

# Exibindo o gráfico
fig_department.show()

####**Distribuição de Gênero por Departamento - Opção B**

In [181]:
import plotly.express as px

# Agrupando o DataFrame pela combinação de Gênero e Departamento e somando o total
df_aggregated = df.groupby(['Genero', 'Departamento']).size().reset_index(name='Total')

# Calculando o total de cada gênero
df_total = df_aggregated.groupby('Genero')['Total'].sum().reset_index()

# Ordenando o DataFrame em ordem decrescente pelo total de cada gênero
df_total_sorted = df_total.sort_values(by='Total', ascending=False)

# Criando o gráfico de barras empilhadas 100%
fig_department = px.bar(df_aggregated, x='Departamento', y='Total', color='Genero',
             title='Distribuição por Departamento e Gênero',
             labels={'Total': 'Total'},
             width=1000, height=600,
             category_orders={'Genero': df_total_sorted['Genero'].unique()},
             barmode='stack', # Definindo o modo de empilhamento
             color_discrete_map={'Fem': 'pink', 'Masc': 'blue'})  # Definindo as cores

# Adicionando os valores nas barras com 2 casas decimais
for trace in fig_department.data:
    trace.text = trace.y.round(2)  # Define o texto para os valores do eixo y com 2 casas decimais

# Adicionando o total de cada barra no topo das barras
for department in df_aggregated['Departamento'].unique():
    total_value = df_aggregated[df_aggregated['Departamento'] == department]['Total'].sum()
    fig_department.add_annotation(
        x=department,
        y=total_value,
        text=f'Total: {total_value}',  # Adiciona o total da barra
        showarrow=False,
        font=dict(size=10),
        xshift=0,
        yshift=10,
    )

# Exibindo o gráfico
fig_department.show()

####**Distribuição de Gênero por Departamento e Senioridade - Opção A**

In [240]:
import math
import plotly.graph_objects as go
import plotly.express as px

# Obtendo a lista de todos os departamentos presentes nos dados
all_departments = df['Departamento'].unique()

# Definindo a ordem das senioridades
seniority_order = ['Estagiário', 'Analista Júnior', 'Analista Pleno', 'Analista Sênior', 'Gerente', 'Diretor']

# Calculando as contagens de departamento e senioridade para o gênero Feminino e Masculino
female_department_seniority_counts = df[df['Genero'] == 'Fem'].groupby(['Departamento', 'Senioridade']).size().unstack(fill_value=0)
female_department_seniority_counts = female_department_seniority_counts.reindex(columns=seniority_order, fill_value=0)
male_department_seniority_counts = df[df['Genero'] == 'Masc'].groupby(['Departamento', 'Senioridade']).size().unstack(fill_value=0)
male_department_seniority_counts = male_department_seniority_counts.reindex(columns=seniority_order, fill_value=0)

# Obtendo o maior valor após o valor máximo observado nas contagens para o nível de senioridade "Estagiário"
max_value_female = female_department_seniority_counts.max().max()
max_value_male = male_department_seniority_counts.max().max()
max_y = math.ceil(max(max_value_female, max_value_male) / 50) * 50

# Criando uma paleta de cores para os departamentos
department_colors = px.colors.qualitative.Set3[:len(all_departments)]

# Criando o gráfico para cada nível de senioridade
fig_department_seniority_female = go.Figure()
fig_department_seniority_male = go.Figure()

# Adicionando os histogramas de distribuição por departamento e senioridade para o gênero Feminino
for department, color in zip(all_departments, department_colors):
    fig_department_seniority_female.add_trace(go.Bar(x=seniority_order, y=female_department_seniority_counts.loc[department], name=f'Fem - {department}', marker=dict(color=color)))

# Adicionando os histogramas de distribuição por departamento e senioridade para o gênero Masculino
for department, color in zip(all_departments, department_colors):
    fig_department_seniority_male.add_trace(go.Bar(x=seniority_order, y=male_department_seniority_counts.loc[department], name=f'Masc - {department}', marker=dict(color=color)))

# Atualizando o layout para mostrar as barras lado a lado e ajustar o eixo y
fig_department_seniority_female.update_layout(title='Distribuição por Departamento e Senioridade - Fem', xaxis_title='Senioridade', yaxis_title='Contagem', yaxis=dict(range=[0, max_y], dtick=50))
fig_department_seniority_male.update_layout(title='Distribuição por Departamento e Senioridade - Masc', xaxis_title='Senioridade', yaxis_title='Contagem', yaxis=dict(range=[0, max_y], dtick=50))

# Exibindo os gráficos
fig_department_seniority_female.show()
fig_department_seniority_male.show()

####**Distribuição de Gênero por Departamento e Senioridade - Opção B**

In [227]:
import math
import plotly.graph_objects as go

# Definindo a ordem das senioridades
seniority_order = ['Estagiário', 'Analista Júnior', 'Analista Pleno', 'Analista Sênior', 'Gerente', 'Diretor']

# Calculando as contagens de departamento e senioridade para o gênero Feminino
female_department_seniority_counts = df[df['Genero'] == 'Fem'].groupby(['Departamento', 'Senioridade']).size().unstack(fill_value=0)
female_department_seniority_counts = female_department_seniority_counts.reindex(columns=seniority_order, fill_value=0)

# Calculando as contagens de departamento e senioridade para o gênero Masculino
male_department_seniority_counts = df[df['Genero'] == 'Masc'].groupby(['Departamento', 'Senioridade']).size().unstack(fill_value=0)
male_department_seniority_counts = male_department_seniority_counts.reindex(columns=seniority_order, fill_value=0)

# Obtendo o maior valor após o valor máximo observado nas contagens para o nível de senioridade "Estagiário"
max_value = max(female_department_seniority_counts.max().max(), male_department_seniority_counts.max().max())
max_y = math.ceil(max_value / 50) * 50

# Criando o gráfico para os níveis de senioridade
fig_department_seniority = go.Figure()

# Adicionando os histogramas de distribuição por departamento e senioridade para o gênero Feminino
for department in df['Departamento'].unique():
    fig_department_seniority.add_trace(go.Bar(x=seniority_order, y=female_department_seniority_counts.loc[department], name=f'Fem - {department}', marker=dict(color='pink'), text=female_department_seniority_counts.loc[department], textposition='auto'))

# Adicionando os histogramas de distribuição por departamento e senioridade para o gênero Masculino
for department in df['Departamento'].unique():
    fig_department_seniority.add_trace(go.Bar(x=seniority_order, y=male_department_seniority_counts.loc[department], name=f'Masc - {department}', marker=dict(color='blue'), text=male_department_seniority_counts.loc[department], textposition='auto'))

# Atualizando o layout para mostrar as barras lado a lado e ajustar o eixo y
fig_department_seniority.update_layout(title='Distribuição por Gênero, Departamento e Senioridade', xaxis_title='Senioridade', yaxis_title='Contagem', yaxis=dict(range=[0, max_y], dtick=50), barmode='group')

# Exibindo o gráfico
fig_department_seniority.show()

####**Distribuição de Gênero por Departamento e Senioridade - Opção C**

In [234]:
import plotly.express as px

# Agrupando os dados
df_defects = df.groupby(['Senioridade', 'Departamento', 'Genero']).size().reset_index(name='Count')

# Criando o gráfico de treemap
fig_treemap = px.treemap(df_defects, path=['Senioridade', 'Departamento', 'Genero'], values='Count',
                 title='Distribuição por Senioridade, Departamento e Gênero - Treemap',
                 labels={'Count': 'Contagem', 'Senioridade': 'Senioridade', 'Departamento': 'Departamento', 'Genero': 'Gênero'},
                 width=1200, height=800)

# Adicionando rótulos de quantidade nos blocos
fig_treemap.update_traces(textinfo='label+value')

# Exibindo o gráfico
fig_treemap.show()

###**Painel Interativo (🚧 em construção)**

In [116]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# Definindo os intervalos de idade
intervalos_idade = [(0, 13), (14, 18), (19, 24), (25, 34), (35, 44), (45, 54), (55, 64), (65, 74), (75, 84), (85, float('inf'))]
labels = ['Menos de 14', '14-18', '19-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84', '85 ou mais']

# Função para contar o número de ocorrências de cada intervalo de idade para uma determinada raça, gênero, formação, tempo de casa e departamento
def contar_ocorrencias_por_intervalo(df, raca, genero, formacao, tempo_de_casa, departamento):
    ocorrencias = []
    for intervalo in intervalos_idade:
        filtro = ((df['Raça'] == raca) & (df['Genero'] == genero) & (df['Formação'] == formacao) & (df['Tempo de casa'] == tempo_de_casa) & (df['Departamento'] == departamento) & (df['Idade'] >= intervalo[0]) & (df['Idade'] <= intervalo[1]))
        ocorrencias.append(filtro.sum())
    return ocorrencias

# Obtendo os valores únicos das colunas 'Raça', 'Gênero', 'Formação', 'Tempo de Casa' e 'Departamento'
racas = df['Raça'].unique()
generos = df['Genero'].unique()
formacoes = df['Formação'].unique()
tempos_de_casa = df['Tempo de casa'].unique()
departamentos = df['Departamento'].unique()

# Criando a aplicação Dash
app = dash.Dash(__name__)

# Layout da aplicação
app.layout = html.Div([
    html.Label('Gênero:'),
    dcc.Dropdown(
        id='genero-dropdown',
        options=[{'label': genero, 'value': genero} for genero in generos],
        value=generos[0]
    ),
    html.Label('Raça:'),
    dcc.Dropdown(
        id='raca-dropdown',
        options=[{'label': raca, 'value': raca} for raca in racas],
        value=racas[0]
    ),
    html.Label('Formação:'),
    dcc.Dropdown(
        id='formacao-dropdown',
        options=[{'label': formacao, 'value': formacao} for formacao in formacoes],
        value=formacoes[0]
    ),
    html.Label('Tempo de Casa:'),
    dcc.Dropdown(
        id='tempo-de-casa-dropdown',
        options=[{'label': tempo, 'value': tempo} for tempo in tempos_de_casa],
        value=tempos_de_casa[0]
    ),
    html.Label('Departamento:'),
    dcc.Dropdown(
        id='departamento-dropdown',
        options=[{'label': departamento, 'value': departamento} for departamento in departamentos],
        value=departamentos[0]
    ),
    dcc.Graph(id='idade-por-filtros')
])

# Callback para atualizar o gráfico com base nos filtros selecionados
@app.callback(
    Output('idade-por-filtros', 'figure'),
    [Input('genero-dropdown', 'value'),
     Input('raca-dropdown', 'value'),
     Input('formacao-dropdown', 'value'),
     Input('tempo-de-casa-dropdown', 'value'),
     Input('departamento-dropdown', 'value')]
)
def update_graph(genero_selecionado, raca_selecionada, formacao_selecionada, tempo_de_casa_selecionado, departamento_selecionado):
    # Contagem de ocorrências por intervalo de idade para a raça, gênero, formação, tempo de casa e departamento selecionados
    ocorrencias = contar_ocorrencias_por_intervalo(df, raca_selecionada, genero_selecionado, formacao_selecionada, tempo_de_casa_selecionado, departamento_selecionado)

    # Criando o gráfico de barras
    fig = go.Figure()

    # Adicionando as barras do gênero selecionado
    fig.add_trace(go.Bar(x=labels, y=ocorrencias, name=genero_selecionado, text=ocorrencias, textposition='outside'))

    # Definindo o layout
    fig.update_layout(
        title='Distribuição de Idade por Filtros',
        xaxis_title='Faixa de Idade',
        yaxis_title='Contagem',
        barmode='group',
        legend_title='Gênero'
    )

    return fig

# Executando a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>